In [ ]:
# to compare various changes in source extraction 
# current changes:
# no background subtraction for FUV 
# upper quartile is min threshold for FUV below .0005 
# no edge mask in threshold / background gen 


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.visualization import ZScaleInterval

In [ ]:
def centile_clip(image, centiles=(1, 95)):
    """
    simple clipping function that clips values above and below a given
    percentile range
    """
    finite = np.ma.masked_invalid(image)
    bounds = np.percentile(finite[~finite.mask].data, centiles)
    result = np.ma.clip(finite, *bounds)
    
    if isinstance(image, np.ma.MaskedArray):
        
        return result
    
    return result.data

In [ ]:
eclipse = 31217
fits_file2 = f'/media/bekah/BekahA/background_test/pics_and_photonlists/fuv_changetest_upperquart/e{eclipse}_FUV_11_30.fits'
fits_file1 = f"/media/bekah/BekahA/background_test/pics_and_photonlists/fuv_nochanges/e{eclipse}_FUV_12_05.fits"

In [ ]:
fig, axs = plt.subplots(10, 4, figsize=(16, 40))
eclipses = ['31217','1640','2614','6307','19188','22118','28781','8549',
           '5985','13338']
for r, eclipse in enumerate(eclipses): 
    fits_file2 = f'/media/bekah/BekahA/background_test/pics_and_photonlists/fuv_changetest_upperquart/e{eclipse}_FUV_11_30.fits'
    fits_file1 = f"/media/bekah/BekahA/background_test/pics_and_photonlists/fuv_nochanges/e{eclipse}_FUV_12_05.fits"

    with fits.open(fits_file1) as hdul1, fits.open(fits_file2) as hdul2:
        data_cnt = hdul2[1].data
        data_src1 = hdul1[2].data
        data_src2 = hdul2[2].data

    binary_src1 = (data_src1 > 0).astype(int)
    binary_src2 = (data_src2 > 0).astype(int)
    source_dif = binary_src1 - binary_src2

    axs[r,0].imshow(centile_clip(data_cnt), origin='lower', cmap='gray', aspect='auto')
    axs[r,0].set_title(f'CNT, eclipse {eclipse}')
    axs[r,0].axis('off')

    axs[r,1].imshow(data_src1!=0, origin='lower', cmap='gray', aspect='auto',interpolation=None)
    axs[r,1].set_title('Sources (orig)')
    axs[r,1].axis('off')

    axs[r,2].imshow(data_src2!=0, origin='lower', cmap='gray', aspect='auto',interpolation=None)
    axs[r,2].set_title('Sources (new min threshold, no bkg sub, no mask)')
    axs[r,2].axis('off')

    axs[r,3].imshow(source_dif!=0, origin='lower', cmap='binary', aspect='auto',interpolation=None)
    axs[r,3].set_title('Source Diff')
    axs[r,3].axis('off')
plt.tight_layout()
plt.show()
plt.savefig("fuv_detection_improvements.png")